In [ ]:
%%pyspark
# Set case sensitive for table and column names
spark.conf.set('spark.sql.caseSensitive', True)

In [ ]:
-- Parâmetros
SET cal.start = 2020-01-01;
SET cal.end   = 2025-12-31;

-- Cria/substitui a tabela gerenciada no Lakehouse (Tables/Calendario)
CREATE OR REPLACE TABLE Calendario
USING delta
AS
WITH dates AS (
  SELECT explode(
           sequence(
             to_date('${cal.start}'),
             to_date('${cal.end}'),
             interval 1 day
           )
         ) AS d
)
SELECT
  CAST(d AS DATE)                                  AS Data,          
  year(d)                                          AS Ano,          
  concat(date_format(d,'yyyy'), ' Q', quarter(d))  AS Trimestre,  -- "YYYY Q1"
  concat(
    date_format(d,'yyyy'), ' ',
    CASE month(d)
      WHEN 1  THEN 'Jan'
      WHEN 2  THEN 'Fev'
      WHEN 3  THEN 'Mar'
      WHEN 4  THEN 'Abr'
      WHEN 5  THEN 'Mai'
      WHEN 6  THEN 'Jun'
      WHEN 7  THEN 'Jul'
      WHEN 8  THEN 'Ago'
      WHEN 9  THEN 'Set'
      WHEN 10 THEN 'Out'
      WHEN 11 THEN 'Nov'
      WHEN 12 THEN 'Dez'
    END
  )                                        AS Mes,    -- "2025 Jan" / "2025 Fev"...
  CAST(date_format(d,'yyyyMM') AS INT)     AS MesOrdinal   -- 202501 etc.
FROM dates;
